# HW 1 - Building a network from scratch

* x1, x2, x3 = (1, 2, -1)
* Weights have been initated with 1. value
* Biases have been initated with 0 value
* The reuqired network must include 2 hidden layers, with 2 neurons in each layer
* One dimension output layer
* true y value = 1


# Loading required packages 

In [2]:
import pandas as pd
import numpy as np
import numpy.typing as npt

### Abstract Class - Layer

In [3]:
from abc import ABC, abstractclassmethod

class Layer(ABC):
    def __init__(self):
        self.input = None
        self.output = None
    
    @abstractclassmethod
    def forward_prop(self, input):
        pass
    
    @abstractclassmethod
    def backward_prop(self, input):
        pass

### Recall - 

    * DE/DX = (DE/DY)*W.T
    * DE/DW = X.T*(DE/DY)
    * DE/DB = DE/DY

### Fully Connected && Activation Layer Classes

In [5]:
class ActivationLayer(Layer):
    def __init__(self, activation, d_activation):
        self.activation = activation
        self.d_activation = d_activation
    
    def forward_prop(self, input_data: npt.ArrayLike) -> npt.ArrayLike:
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    
    def backward_prop(self, output_err: npt.ArrayLike, lr) -> npt.ArrayLike:
        return self.d_activation(self.input) * output_err
    
    
class FullyConnectedLayer(Layer):
    def __init__(self, input_size, output_size, hw_1_init: bool = False):
        if hw_1_init:
            self.weights = np.ones((input_size, output_size)) 
            self.bias = np.zeros((1, 1))
        else:
            self.weights = np.random.rand(input_size, output_size) - 0.5
            self.bias = np.random.rand(1, output_size) - 0.5
        
        
    def forward_prop(self, input_data: npt.ArrayLike) -> npt.ArrayLike:
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
        
    def backward_prop(self, output_err: npt.ArrayLike, lr: float) -> npt.ArrayLike:
        input_err = np.dot(output_err, self.weights.T)
        weights_err = np.dot(self.input.T, output_err)
        
        self.weights -= lr * weights_err
        self.bias -= lr * output_err
        return input_err

### Network

In [6]:
from __future__ import annotations

class Network:
    
    def __init__(self):
        self.layers = []
        self.loss = None
        self.d_loss = None
    
    def add(self, layer: Layer):
        self.layers.append(layer)
    
    def use(self, loss, d_loss):
        self.loss = loss
        self.d_loss = d_loss
    
    def predict(self, input_data: npt.ArrayLike) -> list:
        samples = len(input_data)
        result = []
    
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_prop(output)
            result.append(output)
            
        return result
    
    def fit(self, x_train: npt.ArrayLike, y_train: npt.ArrayLike, epochs: int, lr: float):
        samples = len(x_train)
        
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_prop(output)
                
                err += self.loss(y_train[j], output)
                
                error = self.d_loss(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_prop(error, lr)
            
            err /= samples
            print(f"Epoch {i+1}/{epochs} >> error={err}")
        

### Util Functions

In [7]:
def ReLU(x):
    return np.clip(x, 0, None)

def d_ReLU(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    return (1 - sigmoid(x)) * sigmoid(x)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def d_mse(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

### My Inputs

In [12]:
ACT_FUNCTIONS = {
    'ReLU': (ReLU, d_ReLU),
    'sigmoid': (sigmoid, d_sigmoid)
}

LEARNING_RATE = 0.1
EPOCHS = 3

### Network Inititation - General Example

In [13]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

net = Network()
net.add(FullyConnectedLayer(2, 3))
net.add(ActivationLayer(*ACT_FUNCTIONS['ReLU']))
net.add(FullyConnectedLayer(3, 1))
net.add(ActivationLayer(*ACT_FUNCTIONS['ReLU']))

net.use(mse, d_mse)
net.fit(x_train, y_train, epochs=EPOCHS, lr=LEARNING_RATE)

Epoch 1/3 >> error=0.39409703339126734
Epoch 2/3 >> error=0.35977136247984576
Epoch 3/3 >> error=0.35225563438894236


### Network Inititation - Home Work I

In [14]:
x_train_hw = np.array([[1], [2], [-1]])
y_train_hw = np.array([[0]])


net_hw1 = Network()
net_hw1.add(FullyConnectedLayer(3, 2, hw_1_init=True))
net_hw1.add(ActivationLayer(*ACT_FUNCTIONS['ReLU']))
net_hw1.add(FullyConnectedLayer(2, 2, hw_1_init=True))
net_hw1.add(ActivationLayer(*ACT_FUNCTIONS['ReLU']))

net_hw1.use(mse, d_mse)
# net_hw1.fit(x_train_hw, y_train_hw, epochs=EPOCHS, lr=LEARNING_RATE)